In [5]:
import os, findspark
home=os.path.expanduser("~")
findspark.init(os.path.join(home,"Developer","spark-2.0.0-bin-hadoop2.6"))

In [6]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [7]:
spark.version

u'2.0.0'

In [9]:
import os
#_fp=os.path.join(spark_home,"data/mllib/sample_svm_data.txt")
_fp=os.path.join(os.environ["SPARK_HOME"],\
                 'data','mllib','sample_svm_data.txt')
print os.path.isfile(_fp)

True


In [10]:
_f=open(_fp,'r')
_lines=_f.readlines()
print _lines[0]
_f.close()


1 0 2.52078447201548 0 0 0 2.004684436494304 2.000347299268466 0 2.228387042742021 2.228387042742023 0 0 0 0 0 0



In [11]:
_rdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])
_rdd.take(1)[0]

[1.0,
 0.0,
 2.52078447201548,
 0.0,
 0.0,
 0.0,
 2.004684436494304,
 2.000347299268466,
 0.0,
 2.228387042742021,
 2.228387042742023,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [12]:
from pyspark.mllib.regression import LabeledPoint

_trainRdd0=_rdd.map(lambda line:LabeledPoint(line[0], line[1:]))
_trainRdd0.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

In [13]:
_trainRdd=spark.sparkContext.textFile(_fp)\
    .map(lambda line: [float(x) for x in line.split(' ')])\
    .map(lambda p:LabeledPoint(p[0], p[1:]))
_trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

In [14]:
def createLP(line):
    p = [float(x) for x in line.split(' ')]
    return LabeledPoint(p[0], p[1:])

_rdd=spark.sparkContext.textFile(_fp)
trainRdd = _rdd.map(createLP)

trainRdd.take(1)

[LabeledPoint(1.0, [0.0,2.52078447202,0.0,0.0,0.0,2.00468443649,2.00034729927,0.0,2.22838704274,2.22838704274,0.0,0.0,0.0,0.0,0.0,0.0])]

In [15]:
df = spark.createDataFrame(
    [
        ['No','young', 'false', 'false', 'fair'],
        ['No','young', 'false', 'false', 'good'],
        ['Yes','young', 'true', 'false', 'good'],
        ['Yes','young', 'true', 'true', 'fair'],
        ['No','young', 'false', 'false', 'fair'],
        ['No','middle', 'false', 'false', 'fair'],
        ['No','middle', 'false', 'false', 'good'],
        ['Yes','middle', 'true', 'true', 'good'],
        ['Yes','middle', 'false', 'true', 'excellent'],
        ['Yes','middle', 'false', 'true', 'excellent'],
        ['Yes','old', 'false', 'true', 'excellent'],
        ['Yes','old', 'false', 'true', 'good'],
        ['Yes','old', 'true', 'false', 'good'],
        ['Yes','old', 'true', 'false', 'excellent'],
        ['No','old', 'false', 'false', 'fair'],
    ],
    ['cls','age','f1','f2','f3']
)

In [16]:
df.printSchema()

root
 |-- cls: string (nullable = true)
 |-- age: string (nullable = true)
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- f3: string (nullable = true)



In [17]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

clsIndexer = StringIndexer(inputCol="cls", outputCol="label")
i1Indexer = StringIndexer(inputCol="age", outputCol="i1")
i2Indexer = StringIndexer(inputCol="f1", outputCol="i2")
i3Indexer = StringIndexer(inputCol="f2", outputCol="i3")
i4Indexer = StringIndexer(inputCol="f3", outputCol="i4")
va = VectorAssembler(inputCols=["i1","i2","i3","i4"],outputCol="features")
#df3 = va.transform(df2)

pipeline = Pipeline(stages=[clsIndexer,i1Indexer,i2Indexer,i3Indexer,i4Indexer,va])
# Fit the pipeline to training documents.
model = pipeline.fit(df)
df2 = model.transform(df)

In [18]:
df2.printSchema()
df2.show()

root
 |-- cls: string (nullable = true)
 |-- age: string (nullable = true)
 |-- f1: string (nullable = true)
 |-- f2: string (nullable = true)
 |-- f3: string (nullable = true)
 |-- label: double (nullable = true)
 |-- i1: double (nullable = true)
 |-- i2: double (nullable = true)
 |-- i3: double (nullable = true)
 |-- i4: double (nullable = true)
 |-- features: vector (nullable = true)

+---+------+-----+-----+---------+-----+---+---+---+---+-----------------+
|cls|   age|   f1|   f2|       f3|label| i1| i2| i3| i4|         features|
+---+------+-----+-----+---------+-----+---+---+---+---+-----------------+
| No| young|false|false|     fair|  1.0|0.0|0.0|0.0|1.0|    (4,[3],[1.0])|
| No| young|false|false|     good|  1.0|0.0|0.0|0.0|0.0|        (4,[],[])|
|Yes| young| true|false|     good|  0.0|0.0|1.0|0.0|0.0|    (4,[1],[1.0])|
|Yes| young| true| true|     fair|  0.0|0.0|1.0|1.0|1.0|[0.0,1.0,1.0,1.0]|
| No| young|false|false|     fair|  1.0|0.0|0.0|0.0|1.0|    (4,[3],[1.0])|
| No|midd

In [19]:
trainDf=df2.select('label','features')

In [20]:
trainDf.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [25]:
trainDf.show()

+-----+-----------------+
|label|         features|
+-----+-----------------+
|  1.0|    (4,[3],[1.0])|
|  1.0|        (4,[],[])|
|  0.0|    (4,[1],[1.0])|
|  0.0|[0.0,1.0,1.0,1.0]|
|  1.0|    (4,[3],[1.0])|
|  1.0|[1.0,0.0,0.0,1.0]|
|  1.0|    (4,[0],[1.0])|
|  0.0|[1.0,1.0,1.0,0.0]|
|  0.0|[1.0,0.0,1.0,2.0]|
|  0.0|[1.0,0.0,1.0,2.0]|
|  0.0|[2.0,0.0,1.0,2.0]|
|  0.0|[2.0,0.0,1.0,0.0]|
|  0.0|[2.0,1.0,0.0,0.0]|
|  0.0|[2.0,1.0,0.0,2.0]|
|  1.0|[2.0,0.0,0.0,1.0]|
+-----+-----------------+



In [ ]:
#RDD

In [22]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors
#from pyspark.mllib.util import Vectors

trainRdd = trainDf.rdd.map(lambda row: LabeledPoint(row.label,Vectors.fromML(row.features)))
#trainRdd = trainDf.rdd.map(lambda row: LabeledPoint(row.label,Vectors(row.features)))
trainRdd.collect()

[LabeledPoint(1.0, (4,[3],[1.0])),
 LabeledPoint(1.0, (4,[],[])),
 LabeledPoint(0.0, (4,[1],[1.0])),
 LabeledPoint(0.0, [0.0,1.0,1.0,1.0]),
 LabeledPoint(1.0, (4,[3],[1.0])),
 LabeledPoint(1.0, [1.0,0.0,0.0,1.0]),
 LabeledPoint(1.0, (4,[0],[1.0])),
 LabeledPoint(0.0, [1.0,1.0,1.0,0.0]),
 LabeledPoint(0.0, [1.0,0.0,1.0,2.0]),
 LabeledPoint(0.0, [1.0,0.0,1.0,2.0]),
 LabeledPoint(0.0, [2.0,0.0,1.0,2.0]),
 LabeledPoint(0.0, [2.0,0.0,1.0,0.0]),
 LabeledPoint(0.0, [2.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [2.0,1.0,0.0,2.0]),
 LabeledPoint(1.0, [2.0,0.0,0.0,1.0])]

In [23]:
_trainRdd=trainDf.rdd
print type(_trainRdd)
_trainRdd.collect()

<class 'pyspark.rdd.RDD'>


[Row(label=1.0, features=SparseVector(4, {3: 1.0})),
 Row(label=1.0, features=SparseVector(4, {})),
 Row(label=0.0, features=SparseVector(4, {1: 1.0})),
 Row(label=0.0, features=DenseVector([0.0, 1.0, 1.0, 1.0])),
 Row(label=1.0, features=SparseVector(4, {3: 1.0})),
 Row(label=1.0, features=DenseVector([1.0, 0.0, 0.0, 1.0])),
 Row(label=1.0, features=SparseVector(4, {0: 1.0})),
 Row(label=0.0, features=DenseVector([1.0, 1.0, 1.0, 0.0])),
 Row(label=0.0, features=DenseVector([1.0, 0.0, 1.0, 2.0])),
 Row(label=0.0, features=DenseVector([1.0, 0.0, 1.0, 2.0])),
 Row(label=0.0, features=DenseVector([2.0, 0.0, 1.0, 2.0])),
 Row(label=0.0, features=DenseVector([2.0, 0.0, 1.0, 0.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 0.0, 0.0])),
 Row(label=0.0, features=DenseVector([2.0, 1.0, 0.0, 2.0])),
 Row(label=1.0, features=DenseVector([2.0, 0.0, 0.0, 1.0]))]

In [24]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, regParam=0.01)
lrModel = lr.fit(trainDf)

In [26]:
print lrModel.coefficients
print lrModel.intercept

#feature별로 가중치를 구함
#인터세트는 절편을 구한다고 볼수이있음

[-0.50705810019,-5.31916107407,-5.04694958332,-0.351455356638]
3.2822908185


In [27]:
2*lrModel.coefficients[0]+1*lrModel.coefficients[3]+lrModel.intercept

1.9167192614845543

In [30]:
from pyspark.ml.linalg import Vectors
#from pyspark.mllib.linalg import Vectors,VectorUDT
from pyspark.sql import Row
test0 = spark.sparkContext.parallelize([Row(features=Vectors.dense(2,0,0,1))]).toDF()
result = model1.transform(test0).head()

NameError: name 'model1' is not defined

In [31]:
df = spark.createDataFrame(
    [
        [0,'my dog has flea problems. help please.'],
        [1,'maybe not take him to dog park stupid'],
        [0,'my dalmation is so cute. I love him'],
        [1,'stop posting stupid worthless garbage'],
        [0,'mr licks ate my steak how to stop him'],
        [1,'quit buying worthless dog food stupid'],
        [0,u'우리 강아지 벌레 있어요 도와주세요'],
        [0,u'우리 강아지 귀여워 너 사랑해'],
        [1,u'강아지 공원 가지마 바보같이'],
        [1,u'강아지 음식 구매 마세요 바보같이']
    ],
    ['cls','sent']
)

In [32]:
df.printSchema()

root
 |-- cls: long (nullable = true)
 |-- sent: string (nullable = true)



In [34]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, HashingTF, StopWordsRemover, RegexTokenizer
stopwords=list() 
# 의미없는 단어들 사전을 갱신시켜줌
_mystopwords=[u"나",u"너", u"우리"]
for e in _mystopwords:
    stopwords.append(e)

labelIndexer = StringIndexer(inputCol="cls", outputCol="label")
regexTok = RegexTokenizer(inputCol="sent", outputCol="wordsRegex", pattern="\\s+")#공백으로 스플릿)
#tokenizer = Tokenizer(inputCol="sent", outputCol="words")
stop = StopWordsRemover(inputCol="wordsRegex", outputCol="nostops")
_stopwords=stop.getStopWords()
for e in _stopwords:
    stopwords.append(e)
stop.setStopWords(stopwords)

hashingTF = HashingTF(inputCol="nostops", outputCol="features")
pipeline = Pipeline(stages=[labelIndexer,regexTok,stop,hashingTF])
model=pipeline.fit(df)
trainDf = model.transform(df)

In [35]:
trainDf.select('cls','label','features').show()


+---+-----+--------------------+
|cls|label|            features|
+---+-----+--------------------+
|  0|  0.0|(262144,[25688,38...|
|  1|  1.0|(262144,[55639,71...|
|  0|  0.0|(262144,[46165,81...|
|  1|  1.0|(262144,[57368,87...|
|  0|  0.0|(262144,[69384,88...|
|  1|  1.0|(262144,[57368,75...|
|  0|  0.0|(262144,[14050,28...|
|  0|  0.0|(262144,[28575,57...|
|  1|  1.0|(262144,[28575,17...|
|  1|  1.0|(262144,[28575,84...|
+---+-----+--------------------+



In [36]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.linalg import Vectors

trainRdd = trainDf\
    .rdd\
    .map(lambda row: LabeledPoint(row.label,Vectors.fromML(row.features)))
trainRdd.take(1)

[LabeledPoint(0.0, (262144,[25688,38977,75919,87576,239859],[1.0,1.0,1.0,1.0,1.0]))]

In [37]:
from pyspark.ml.classification import *
lr = LogisticRegression(maxIter=10, regParam=0.01)
lrModel = lr.fit(trainDf)

In [38]:
print "* summary: ", lrModel.summary
print "* coefficients: ", lrModel.coefficients
print "* intercept: ", lrModel.intercept

* summary:  <pyspark.ml.classification.BinaryLogisticRegressionTrainingSummary object at 0x1099cc7d0>
* coefficients:  (262144,[14050,25688,28575,38977,46165,55639,57298,57368,69384,71826,75919,81486,84127,87345,87576,88312,103148,104257,121133,121786,158088,163197,173036,175143,186022,186480,202268,220548,228185,239859,243184,244249,245237,250806,251329],[-1.27222161806,-1.0959340667,-0.0755352913925,-1.0959340667,-1.29204386194,1.13289974307,-1.75263828924,1.16070634922,-1.0381881797,1.13289974307,0.376841766526,-1.29204386194,1.19058324228,1.22114617531,-1.0959340667,-1.0381881797,0.842331778855,-1.27222161806,0.842331778855,1.19058324228,-1.75263828924,1.22114617531,1.6328492213,1.13289974307,0.842331778855,-1.29204386194,0.102913872536,-1.0381881797,1.58818076076,-1.0959340667,-1.27222161806,-1.0381881797,1.19058324228,1.6328492213,1.36987615596])
* intercept:  -0.596615892431


In [ ]:
from operator import add
wcReduceByKey = wc.reduceByKey(add)
In [19]:
wcGroupByKey = wc.groupByKey().mapValues(sum)
In [24]:
wcGroupByKey2 = wc.groupByKey().map(lambda (x,v): (x,len(v)))
In [32]:
wcReduceByKey.sortByKey().take(10)


#키별로 섭 워드벡터로 가운팅